In [13]:
from dataloader import DataLoader
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf

In [11]:
dl = DataLoader()
X_train, y_train, X_test, y_test = dl.load_mnist()
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
def convert_labels_to_binary(labels):
    return np.where(labels % 2 == 0, 0, 1)


y_train_binary = convert_labels_to_binary(y_train)
y_test_binary = convert_labels_to_binary(y_test)

print("First 10 binary labels in training set:", y_train_binary[:10])
print("First 10 binary labels in test set:", y_test_binary[:10])

First 10 binary labels in training set: [1 0 0 1 1 0 1 1 1 0]
First 10 binary labels in test set: [1 0 1 0 0 1 0 1 1 1]


In [14]:

# Define the policy network
def build_policy_network():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(2, activation='softmax')  # Outputs the probability of being positive or negative
    ])
    return model

# Define the classifier
def build_classifier():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(1, activation='sigmoid')  # Binary classifier
    ])
    return model

policy_network = build_policy_network()
classifier = build_classifier()

C:\Users\FH-Deg\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [16]:
# Compile the models
policy_network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



def train_policy_network(policy_network, classifier, X_train, y_train_binary):
    # Parameters
    learning_rate = 1e-3
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    with tf.GradientTape() as tape:
        # 1. Use the policy network to predict actions (pseudo-labels) for the training data
        actions = policy_network(X_train, training=True)

        # 2. Convert actions to pseudo-labels. This might involve taking the argmax if your actions are categorical.
        pseudo_labels = tf.argmax(actions, axis=1)

        # 3. Combine true labels and pseudo-labels
        # For simplicity, we're treating all data as unlabeled here; you might have a more sophisticated method
        combined_labels = tf.where(y_train_binary >= 0, y_train_binary, pseudo_labels)

        # 4. Train the classifier on the combined dataset and calculate loss
        classifier_predictions = classifier(X_train, training=True)
        # Squeeze the predictions to match the target's shape
        classifier_predictions = tf.squeeze(classifier_predictions, axis=-1)
        classifier_loss = tf.keras.losses.binary_crossentropy(combined_labels, classifier_predictions)

    # Calculate gradients with respect to policy network's parameters
    gradients = tape.gradient(classifier_loss, policy_network.trainable_variables)

    # Apply gradients to update the policy network
    optimizer.apply_gradients(zip(gradients, policy_network.trainable_variables))


# Training loop
epochs = 10  # Example number of epochs

for epoch in range(epochs):
    # Train the classifier on the original labeled data
    classifier.fit(X_train, y_train_binary, epochs=1, batch_size=32, verbose=1)
    
    # Update the policy network based on the classifier's performance
    train_policy_network(policy_network, classifier, X_train, y_train_binary)

    print(f'Epoch {epoch+1}/{epochs} completed.')

# Test the classifier
test_loss, test_acc = classifier.evaluate(X_test, y_test_binary, verbose=2)
print(f'\nTest accuracy: {test_acc}')


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9649 - loss: 0.1009


ValueError: No gradients provided for any variable.